In [2]:
import numpy as np
import random
import copy
from collections import deque
import pandas as pd

In [6]:

# Define the CVRP problem data
class CVRP:
    def __init__(self, num_nodes, num_vehicles, capacity):
        self.num_nodes = num_nodes
        self.num_vehicles = num_vehicles
        self.capacity = capacity
        self.distance_matrix = [[0] * num_nodes for _ in range(num_nodes)]

def create_cvrp_instance(num_nodes, num_vehicles, capacity):
    cvrp = CVRP(num_nodes, num_vehicles, capacity)

    # Randomly generate distances (symmetric) between nodes
    for i in range(num_nodes):
        for j in range(i, num_nodes):
            if i != j:
                distance = random.randint(1, 20)  # Adjust the range as needed
                cvrp.distance_matrix[i][j] = distance
                cvrp.distance_matrix[j][i] = distance

    return cvrp

def calculate_route_cost(route, cvrp):
    cost = 0
    demand = 0
    for i in range(len(route) - 1):
        cost += cvrp.distance_matrix[route[i]][route[i+1]]
        demand += route[i]
        if demand > cvrp.capacity:
            return float('inf')  # Infeasible solution
    cost += cvrp.distance_matrix[route[-1]][0]  # Return to depot
    return cost

def tabu_search(cvrp, max_iterations, tabu_size):
    best_solution = list(range(1, cvrp.num_nodes))  # Initial solution
    random.shuffle(best_solution)
    best_solution_cost = calculate_route_cost([0] + best_solution + [0], cvrp)

    current_solution = best_solution
    current_solution_cost = best_solution_cost

    tabu_list = deque(maxlen=tabu_size)

    for _ in range(max_iterations):
        neighborhood = []
        for i in range(1, len(current_solution) - 1):
            for j in range(i + 1, len(current_solution)):
                neighbor = copy.copy(current_solution)
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                neighbor_cost = calculate_route_cost([0] + neighbor + [0], cvrp)
                neighborhood.append((neighbor, neighbor_cost))

        neighborhood.sort(key=lambda x: x[1])
        for neighbor, neighbor_cost in neighborhood:
            if neighbor_cost < current_solution_cost and neighbor not in tabu_list:
                current_solution = neighbor
                current_solution_cost = neighbor_cost
                if current_solution_cost < best_solution_cost:
                    best_solution = current_solution
                    best_solution_cost = current_solution_cost
                tabu_list.append(neighbor)
                break

    return best_solution, best_solution_cost

def plot_cvrp_solution(cvrp, solution):
    G = nx.Graph()

    # Add depot node
    G.add_node(0, pos=(0, 0), demand=0)

    # Add customer nodes
    for i, node in enumerate(solution):
        G.add_node(i + 1, pos=(random.uniform(1, 10), random.uniform(1, 10)), demand=node)

    # Add edges
    for i in range(len(solution)):
        G.add_edge(0, i + 1)

    pos = nx.get_node_attributes(G, 'pos')
    demand = nx.get_node_attributes(G, 'demand')

    nx.draw(G, pos, with_labels=True, node_size=300, node_color='lightblue')
    nx.draw_networkx_labels(G, pos)
    nx.draw_networkx_edge_labels(G, pos, edge_labels=demand, font_color='red')
    plt.axis('off')
    plt.show()

def main():
    num_nodes = 20  # Number of nodes (including the depot)
    num_vehicles = 4  # Number of vehicles
    capacity = 100  # Vehicle capacity
    max_iterations = 1000
    tabu_size = 15

    cvrp = create_cvrp_instance(num_nodes, num_vehicles, capacity)

    best_solution, best_solution_cost = tabu_search(cvrp, max_iterations, tabu_size)

    print("Best solution:", [0] + best_solution + [0])
    print("Best solution cost:", best_solution_cost)

if __name__ == "__main__":
    main()


Best solution: [0, 9, 17, 11, 3, 8, 13, 1, 6, 15, 16, 5, 7, 12, 10, 4, 2, 18, 14, 19, 0]
Best solution cost: inf
